In [89]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **문맥 압축 검색기(ContextualCompressionRetriever)**

`Contextual`(맥락과 관련된), `Compression`(압축), `Retriever(검색기)` <br>

검색된 문서를 그대로 즉시 반환하는 대신, 주어진 질의의 맥락을 사용하여 문서를 압축함으로써 관련 정보만 반환되도록 할 수 있습니다.<br>
검색된 문서에서 압축을 해서 꼭 필요한 정보만 (질문과 연관성이 있는 정보) 추려서 k에 넣겠다 라는 것입니다. <br><br>
LLM 기반 압축기, 임베딩 기반 압축기가 있습니다. <br><br>

`ContextualCompressionRetriever` 는 질의를 `base retriever`에 전달하고, 초기 문서를 가져와 `Document Compressor`를 통과시킵니다. <br>

In [90]:
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        print(f"문서 {i+1}:\n\n" + d.page_content)
        print(f"{'-' * 100}\n")

### 기본 Retriever 설정

벡터 스토어 retriever를 초기화하고 텍스트 문서를 청크 단위로 저장합니다. <br>
질문을 던졌을 때, retriever는 관련 있는 문서 1~2개와 관련 없는 문서 몇 개를 반환하는 것을 확인할 수 있습니다.

In [91]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [92]:
loader = TextLoader("./data/appendix-keywords.txt", encoding='utf-8')

In [93]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

In [94]:
embeddings = OpenAIEmbeddings(api_key=key)

In [95]:
# as_retriever()에 기본값으로 k는 4개로 설정되어있기 때문에 문서 4개를 검색에 포함해서 가져온다.
retriever = FAISS.from_documents(texts, embedding=embeddings).as_retriever()

In [96]:
# 쿼리에 질문을 정의하고 관련 문서를 검색합니다.
docs = retriever.invoke("Semantic Search 에 대해서 알려줘.")

In [97]:
pretty_print_docs(docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------

문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------

문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------

문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은

## **맥락적 압축(ContextualCompression)**

`LLMChainExtractor` 를 활용하여 생성한 `DocumentCompressor` 를 `retriever` 에 적용한 것이 바로 `ContextualCompressionRetriever` 입니다.

In [98]:
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

In [99]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  

In [100]:
# LLM을 사용하여 문서 압축기 생성
compressor = LLMChainExtractor.from_llm(llm)

In [101]:
# 문서 압축기와 리트리버를 사용하여 컨텍스트 압축 리트리버 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,         # LLM을 사용하여 만든 문서 압축기 지정
    base_retriever=retriever,           # 위에서 만든 기본 retriever를 지정
)

일반 retriever로 invoke를 해서 나온 결과와 압축기로 검색한 결과를 비교해 본다.

In [102]:
# 일반 retriever 검색.
docs = retriever.invoke("Semantic Search 에 대해서 알려줘.")

In [103]:
pretty_print_docs(docs)     # 문서 4개가 나온다.

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------

문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------

문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------

문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은

압축기로 검색

In [104]:
# 컨텍스트 압축 리트리버를 사용하여 관련 문서 검색
compressed_docs = (
    compression_retriever.invoke( 
        "Semantic Search 에 대해서 알려줘."
    )
)

In [105]:
pretty_print_docs(compressed_docs)      # 질문과 관련된 문서만 나온다.

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
----------------------------------------------------------------------------------------------------



### LLM 을 활용한 문서 필터링

`LLMChainFilter`는 문서 필터링 기능을 합니다. 질문과 관련성이 있는 높은 문서들만 필터링하고 관련성이 없는 문서는 drop. <br>
이 필터는 문서 내용을 변경(압축)하지 않고 문서를 선택적으로 반환 합니다.

In [106]:
from langchain_teddynote.document_compressors import LLMChainFilter

In [107]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  

In [108]:
# LLM을 사용하여 LLMChainFilter 객체를 생성
_filter = LLMChainFilter.from_llm(llm)

In [109]:
# ContextualCompressionRetriever 객체를 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter,        # 위에서 만든 LLMChainFilter
    base_retriever=retriever,       # 위에서 만든 retriever    
)

In [110]:
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

In [111]:
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------



### **EmbeddingsFilter**

`EmbeddingsFilter`는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.<br>

`EmbeddingsFilter`는 `embedding` 기반으로 필터합니다. <br>
embedding 기반으로 필터를 할때에 유사도를 다시 한번 계산해서 similarity_threshold 값을 주고 그 값(threshold) 이상인 것들만 필터링을 걸게 됩니다. <br>

In [112]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

In [113]:
embeddings = OpenAIEmbeddings(api_key=key)

In [114]:
# 유사도 임계값이 0.86인 EmbeddingsFilter 객체를 생성
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

In [115]:
# ContextualCompressionRetriever 객체를 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,  # 기본 압축기로 embeddings_filter를 지정
    base_retriever=retriever            # 기본 검색기로 retriever 지정
)

In [116]:
# ContextualCompressionRetriever 객체를 사용하여 관련 문서를 검색합니다.
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

In [117]:
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------



### **파이프라인 생성(압축기+문서 변환기)**

DocumentCompressorPipeline 을 사용하면 여러 compressor를 순차적으로 결합할 수 있습니다.<br>
절차를 순서로 매겨서 순서대로 실행 되도록 만들어 줄수가 있습니다. <br>

In [118]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

In [125]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  

In [119]:
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

In [120]:
# 임베딩을 사용하여 redundant_filter(중복 필터)를 생성합니다.
# 검색된 문서 사이에 유사도(0.95으로 설정됨) 계산을 하고 유사도가 0.95 보다 높은 것들은 중복으로 판단해서 drop
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

In [121]:
# 임베딩을 사용하여 관련성 필터를 생성하고, 유사도 임계값을 0.86으로 설정합니다.
# 유사도 기반으로 0.86 이상인 것들은 걸러준다.
# 관련성 있는 문서만 있도록 필터링한다.
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

In [122]:
# 먼저 문서를 더 작은 청크로 분할한 다음, 중복 문서를 제거하고, 
# 쿼리와의 관련성을 기준으로 필터링하여 compressor pipeline을 생성

In [123]:
pipeline_compressor = DocumentCompressorPipeline(
    # 문서 압축 파이프라인을 생성하고, 분할기, 중복 필터, 관련성 필터, LLMChainExtractor를 변환기로 설정합니다.
    transformers=[
        splitter,               # 분할기 (문서를 조각들로 쪼갠다)
        redundant_filter,       # 중복되는 문서 걸러주는 필터.
        relevant_filter,        # 관련성 있는 문서만 있도록 걸러주는 필터. embeddingsFilter(유사도 기반으로 0.86 이상인 것들은 걸러주는 필터).
        LLMChainExtractor.from_llm(llm),  # 압축
    ]
)

`ContextualCompressionRetriever`를 초기화하며, `base_compressor`로 `pipeline_compressor`를, `base_retriever`로 `retriever`를 사용합니다.

In [124]:
# 기본 압축기로 pipeline_compressor를 사용하고, 
# 기본 검색기로 retriever를 사용하여 ContextualCompressionRetriever를 초기화합니다.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,    # 기본 압축기로 pipeline_compressor를 사용
    base_retriever=retriever,               # 기본 검색기로 retriever
)

In [126]:
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

In [127]:
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
----------------------------------------------------------------------------------------------------

